In [103]:
import copy

import pymdp
from pymdp import utils
from pymdp.agent import Agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
""" Specify the dimensionalities of the hidden state factors, the control factors, and the observation modalities """

' Specify the dimensionalities of the hidden state factors, the control factors, and the observation modalities '

In [92]:
trials = 64
T = 2  # number of timepoints per trial

#STATES
# state if the img is cs+ or cs-
state = ['cs+', 'cs-']


#OBSERVATIONS

# morphing level binary. shock only if morph_level=6
morph_level_obs = ['1','6']
# the subject receive the electric shock?
shock_obs = ['shock', 'no shock']
# observation to discriminate if the subject is surprised or prepared. A sort of diff between user prediction and shock observation
surprised_obs = ['not_surprised','surprised']

#ACTIONS
# the subject predict if should be receive the shock or not
action = ['guess-cs+','guess-cs-']

# Define `num_states` and `num_factors` below
num_states = [len(state)] #[2]
num_factors = len(num_states) #1

#Define `num_obs` and `num_modalities` below
num_obs = [len(morph_level_obs), len(shock_obs), len(surprised_obs)] # [2,2,2]
num_modalities = len(num_obs) #3



In [93]:
num_factors

1

In [104]:
def matrices(TrialEnv):
    """
    The A, B, C, D, E matrices are built.
    """
    # This is the likelihood mapping that truly describes the relationship between the
    # environment's hidden state and the observations the agent will get

    # A matrix
    # A dimension?
    A = np.zeros((2, len(state),num_modalities))

    #mor_lev_binary/states
    A[:, :, 0] = np.array([ [0, 1],
                            [1, 0] ]
                          )

    #shock-noshock/states
    A[:, :, 1] = np.array([ [0.75, 0],
                            [0.25, 1] ]
                          )

    #prepared-surprised/states
    A[:, :, 2] = np.array([ [0.5, 0.5],
                            [0.5, 0.5] ]
                          )

    # B matrix
    # The transition mapping that truly describes the environment's dynamics

    #B as an identity matrix. no state transitions
    B = utils.obj_array(num_factors)
    B_ = np.zeros((1, len(state), len(state)))

    B_[0, :,:] = np.array([ [1, 0],
                           [0, 1] ])

    B[0]=B_
    B_gm = copy.deepcopy(B)  # make a copy of the true transition likelihood to initialize the transition model
    #???
    #A_gm = copy.deepcopy(A_gp)  # make a copy of the true observation likelihood to initialize the observation model
    #???
    #B_gm = copy.deepcopy(B_gp)  # make a copy of the true transition likelihood to initialize the transition model


    # This is the matrix representing the preferences over outcomes
    #prepared-surprised/cs+-
    C = [4.0,-6.0]

    return A, B_gm, C

In [39]:
len(state)

2

In [19]:
""" Define `num_states` and `num_factors` below """
num_states = [len(context_names), len(choice_names)]
num_factors = len(num_states)

context_action_names = ['Do-nothing']
choice_action_names = ['guess-cs+','guess-cs-']

""" Define `num_controls` below """
num_controls = [len(context_action_names), len(choice_action_names)]

hint_obs_names = ['Null', 'Hint-left', 'Hint-right']
reward_obs_names = ['Null', 'Loss', 'Reward']
choice_obs_names = ['Start', 'Hint', 'Left Arm', 'Right Arm']

""" Define `num_obs` and `num_modalities` below """
num_obs = [len(hint_obs_names), len(reward_obs_names), len(choice_obs_names)]
num_modalities = len(num_obs)

In [20]:
df = data
# num_trial # cs+ / cs-

NameError: name 'data' is not defined

In [21]:
def get_reward(shock_obs, surprised_obs, action):
    # TODO how can implement this?
    diff = abs(surprised_obs-shock_obs)
    if diff == 0:
        reward = +4
    else:
        reward = -6
    return reward


def get_reward_predicted_low(shock_obs, surprised_obs):
    if surprised_obs == 1: # subject is surprised and reward is
        reward = -6
    else:
        reward = 4


def get_reward_predicted_high(shock_obs, surprised_obs):
    if surprised_obs == 1:
        reward = -6
    else:
        reward = 4


class TrialEnv(object):

  def __init__(self,data):
    self.state = 0
    self.data = data #num_trial #cs+,cs-

  def get_observation(self,param):
    obs = None
    if param == 'morph_level':
        if self.data == 'cs+':
            obs=6
        else:
            obs=1

    if param == 'shock':
        if self.data == 'cs+':
            obs=0.75
        else:
            obs=0.25

    if obs is None:
        print('key error get observation')

    return obs


  def choose_action(self):

      # Take a step in the environment given an action
      looked_mor_lev = self.get_observation(param='morph_level')

      if looked_mor_lev == 6:
          action = 'guess cs+'

      if looked_mor_lev == 1:
          action = 'guess cs-'

      return action

  def step(self, action):
      # Take a step in the environment given an action

      if action == 'guess cs+':
          self.state=0

      if action == 'guess cs-':
          self.state=1


      #action_done = (guess cs+)
      if self.state==0:
          shock_obs = self.get_observation(param='shock')

          if shock_obs >0.5: # in this case shock and i have predicted cs+ #TODO change this
            surprised_obs = self.get_surprised_low()

          else: # in this case no shock and i have predicted cs+
            surprised_obs = self.get_surprised_high()

          reward = get_reward_predicted_high(shock_obs,surprised_obs)

      #action_done = (guess cs-)
      if self.state == 1:
          shock_obs = self.get_observation(param='shock')

          if shock_obs is high: # in this case shock and i have predicted cs-
            surprised_obs = self.get_surprised_high()

          else: # in this case no shock and i have predicted cs-
            surprised_obs = self.get_surprised_low()


          reward = get_reward_predicted_low(shock_obs,surprised_obs)


      obs = [morph_level_obs, shock_obs, surprised_obs]


      return obs,reward


  def get_surprised_high(self):
      return 1

  def get_surprised_low(self):
      return 0


In [105]:
A,B,C = matrices(TrialEnv)

In [106]:
A

array([[[0.  , 0.75, 0.5 ],
        [1.  , 0.  , 0.5 ]],

       [[1.  , 0.25, 0.5 ],
        [0.  , 1.  , 0.5 ]]])

In [107]:
[B[f].shape[2] for f in range(1)]

[2]

In [108]:
B

array([array([[[1., 0.],
               [0., 1.]]])], dtype=object)

In [109]:
C

[4.0, -6.0]

In [110]:
B.sum(axis = 0)

array([[[1., 0.],
        [0., 1.]]])

In [111]:
# define an agent
my_agent = Agent(A = A, B = B, C = C)

# define an environment
my_env = TrialEnv()

initial_action = 'first_action'

# set an initial action
action = initial_action

# iterate over all trials
for t in range (trials) :

    action = my_env.choose_action()

    obs,reward = my_env.step(action)

    my_agent.infer_states (obs) # agent update beliefs about hidden states given observations

    my_agent.infer_policies () # inferring policies and sampling actions from the posterior

    action = my_agent.sample_action()

AssertionError: B matrix is not normalized (i.e. B.sum(axis = 0) must all equal 1.0)

In [57]:
B.shape

(1, 2, 2)

In [60]:
B[0].shape[2]

IndexError: tuple index out of range

In [71]:
B_locs = np.eye(2).reshape(2,2,1)
B_locs

array([[[1.],
        [0.]],

       [[0.],
        [1.]]])

In [72]:
B_locs = np.tile(B_locs, (1, 1, 2))

In [73]:
B_locs

array([[[1., 1.],
        [0., 0.]],

       [[0., 0.],
        [1., 1.]]])

In [75]:
B_locs.shape

(2, 2, 2)

In [76]:
 B_locs = B_locs.transpose(1, 2, 0)

In [77]:
B_locs

array([[[1., 0.],
        [1., 0.]],

       [[0., 1.],
        [0., 1.]]])